## Used Cars Market: Web Scraping

### Purpose the study

The Used cars market in the past few years has reached a big improvement in the number of Buyers, Sellers and Investors due to the Economic developments that the country has gone through and the nature of the market it self.

This has raised alot of questions about the market and the concern to estimate the different types of used cars prices.

For that purpose i am conducting a Data Science project that would answer these questions and predict the cars prices in the 
Egyptian market.

This project will go through Five main stages:

1. Getting Used Cars data (Scraping data)
2. Data Cleaning
3. Features Engineering
4. Data Anlaysis
5. Applying Machine Learning Model

Objective of this Notebook:

In this notebook i will go through the first stage of my project which is scraping data, i am going to scrape around 12000 Ads from www.Contactcars.com website to use them in my project.

In [17]:
#Importing libraries
import pandas as pd
import time
from bs4 import BeautifulSoup as BS
import requests
import re
import selenium
from selenium import webdriver
import numpy as np

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from itertools import product

In [18]:
# I kept all the values of the cars details to use them in our scraper later on
body_type_values=['Cabriolet','Coupe','Hatchback','SUV','Sedan','Station','Van']
transmission_type_values=['CVT','Dual Clutch','Manual','Automatic']
motor_capacity_values=['800 cc','900 cc','1000 cc','1100 cc','1200 cc','1300 cc','1400 cc','1500 cc','1600 cc','1700 cc','1800 cc','1900 cc','2000 cc','2200 cc','2400 cc','2500 cc','2700 cc','2800 cc','2900 cc','3000 cc','3200 cc','3400 cc','3500 cc','3600 cc','3700 cc','3800 cc','3900 c','4000 cc','4200 cc','4300 cc','4400 cc','4600 cc','4700 cc','4800 cc','5000 cc','5400 cc','5600 cc','5700 cc','5800 cc','6000 cc','6200 cc','6400 cc']
color_values=['Baby Blue','Beige','Black','Blue','Bronze','Brown','Champagne','Dark Grey','Gold','Gray','Green','Maroon','Navy Blue','Orange','Pink','Purple','Red','Silver','Turquoise','White','Yellow']

In [19]:
Cars_links = []
def collect_add_links(page_nums):
    driver=webdriver.Chrome(r"C:\Users\abdal\Downloads\Contact cars - WEB SCARAPING\chromedriver.exe")
            
    # Intialize
    print('Starting, please wait....')
    
    for page_num in list(range(page_nums + 1))[1:]:
        # Get page link
        link = f'https://www.contactcars.com/en/cars/used/engines?make=120&make=1&make=64&make=33&make=2&make=68&make=63&make=42&make=109&make=61&make=115&make=59&make=122&make=57&make=21&make=22&make=23&make=81&make=101&make=11&make=30&make=36&make=45&make=116&make=6&make=24&make=102&make=58&make=52&make=53&make=123&make=108&make=119&make=25&make=46&make=7&make=43&make=73&make=66&make=124&make=31&make=27&make=117&make=10&make=51&make=26&make=103&make=38&make=37&make=113&make=48&make=56&make=50&make=110&make=62&make=28&make=3&make=47&make=39&make=4&make=29&make=14&make=15&make=34&make=16&make=107&make=40&make=32&make=55&make=17&make=69&make=104&make=118&make=105&make=8&make=65&make=19&make=70&make=60&make=44&make=41&make=35&make=20&make=121&make=5&make=13&make=12&make=114&page={page_num}&sortOrder=false&sortBy=CreatedAt'
        driver.get(link)
        
        # sleep for 2 seconds after loading the page
        time.sleep(1)
        
        # working on page number: 
        print(f'Working in page {page_num}....')
        
        # Scrolling fuction
        scroll_pause_time = 1
        screen_height = driver.execute_script("return window.screen.height;")

        i = 1 
        while True:
            elems=driver.find_elements_by_css_selector('div.full-side')
            driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
            i += 1
            time.sleep(scroll_pause_time)
            # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
            scroll_height = driver.execute_script("return document.body.scrollHeight;")  
            # Break the loop when the height we need to scroll to is larger than the total scroll height
            if (screen_height) * i > scroll_height:
                break
                
        # sleep for 2 seconds after scrolling down the page        
        time.sleep(1)
        
        # Get page html content 
        src = driver.page_source
        soup=BS(src,'html.parser')

        # Get all adds  
        Cars_link = soup.find_all('div', {'class':['col-xl-4 col-md-6 mt-3 px-2 ng-star-inserted']})

        # Get links from these adds 
        for item in Cars_link: 
            for link in item.find_all('a', href = True):
                Cars_links.append(link["href"])

        print(f'Page {page_num} completed')
        
        if page_num == page_nums:
            print(f'Job Done: All {len(Cars_links)} Links from your {page_nums} have been collected')

In [20]:
collect_add_links(1)

Starting, please wait....
Working in page 1....
Page 1 completed
Job Done: All 48 Links from your 1 have been collected


In [21]:
# A sample of the collected Cars links
Cars_links[:4]

['/en/car/61cc9687e85ee0656fca4af5-Mercedes-E350-3.5-A-T-AMG?status=Used',
 '/en/car/61cc8e597417f5d39f2d42be-Mercedes-E300-1.9-Avantgarde?status=Used',
 '/en/car/61cc8ce3a8535a4581752566-Mercedes-GLC-300--2.0-A-T-Coupe-AMG?status=Used',
 '/en/car/61cc88abe85ee0656fca4a3c-Mercedes-GLA-200-1.3-A-T-Progressive?status=Used']

In [22]:
#These are all the features i am going to scrap from the website
Mobile_number=[]
Model_name=[]
SubModel_name=[]
Car_condition=[]
Kilometers_crossed=[]
Car_price=[]
Seller_City=[]
Model_year=[]
body_type=[]
Transmission_type=[]
Motor_capacity=[]
Car_color=[]
waranty_condition=[]
Car_paint=[]
Ad_post_date=[]
Interior_Features=[]
Exterior_Features=[]
Multimedia_Features=[]
Safety_Features=[]

In [23]:
ADs_list = []
starting_part = 'https://www.contactcars.com'
count = 1

for link in Cars_links[:10]:
    print(f'AD number {count}: Collecting data....')
    driver.get(starting_part+link)
    time.sleep(1)
    html=driver.page_source #import page source
    soup=BS(html,'html.parser')

    
    #-----------------------------------------
    #section 1: easy straight forward features
    #----------------------------------------- 
    
    
    try:
          Model_name.append(soup.find_all('a',{'class':'breadcrumb__items__item ng-star-inserted'})[1].text.strip())
    except AttributeError:
          Model_name.append('Not available')
    try:
          Kilometers_crossed.append(soup.find('div',{'class':'car-details__status ng-star-inserted'}).text.strip())
    except AttributeError:
          Kilometers_crossed.append('Not available')
    try:
          SubModel_name.append(soup.find('div',{'class':'d-inline-block'}).text.strip())
    except AttributeError:
          SubModel_name.append('Not available')
    try:
          Car_price.append(soup.find('h3',{'class':'car-details__used-info__car-info__finance__total'}).text.strip())
    except AttributeError:
          Car_price.append('Not available')
    try:
          Car_condition.append(soup.find('p',{'class':'car-details__used-info__about__details pre-line'}).text.strip())
    except AttributeError:
          Car_condition.append('Not available')
    try:
          Seller_City.append(soup.find('div',{'class':'car-details__used-info__car-info__locations ng-star-inserted'}).text.strip())
    except AttributeError:
          Seller_City.append('Not available')
    try:
          Model_year.append(soup.find('div',{'class':'d-inline-block margin-start--sm'}).text.strip())
    except AttributeError:
          Model_year.append('Not available')
            
        
    Date_div = soup.find_all("span", class_="car-details__sumary__item__details")
        
    try:
          for i in Date_div[3]:
            Ad_post_date.append(i.strip()) # we chose Div[2] because it has the UsedCar_post_date in it
    except IndexError:
                    Ad_post_date.append('Not available')
            
            
                
    #-----------------------------------------
    #section 2: Getting Additional Features from the ads
    #----------------------------------------- 
    

    try:
          Interior_Features.append(soup.find_all('div',{'class':'row mx-n5'})[0].text)
    except IndexError:
          Interior_Features.append('Not available')
            
    try:
          Exterior_Features.append(soup.find_all('div',{'class':'row mx-n5'})[1].text)
    except IndexError:
          Exterior_Features.append('Not available')
            
    try:
          Multimedia_Features.append(soup.find_all('div',{'class':'row mx-n5'})[2].text)
    except IndexError:
          Multimedia_Features.append('Not available')
            
    try:
          Safety_Features.append(soup.find_all('div',{'class':'row mx-n5'})[3].text)
    except IndexError:
          Safety_Features.append('Not available')
        
        
    #-----------------------------------------
    #section 3: Getting car details such as (color, bodytype, motor capacity, ...) 
    #----------------------------------------- 
    
    
    try:
        car_details = soup.find_all('span', class_ = 'car-details__car-data__item__desc')
        car_details_list = []
        for item in car_details:
            car_details_list.append(item.text.strip())

    
        if set(color_values)&set(car_details_list):
            Car_color.append(','.join(set(color_values)&set(car_details_list)))
        else:
            Car_color.append('Not available')
                
     
        if set(body_type_values)&set(car_details_list):
            body_type.append(','.join(set(body_type_values)&set(car_details_list)))
        else:
            body_type.append('Not available')
            
            
        if set(transmission_type_values)&set(car_details_list):
            Transmission_type.append(','.join(set(transmission_type_values)&set(car_details_list)))
        else:
            Transmission_type.append('Not available')
            
            
        if set(motor_capacity_values)&set(car_details_list):
            Motor_capacity.append(','.join(set(motor_capacity_values)&set(car_details_list)))
        else:
            Motor_capacity.append('Not available')
            
            
            
        if 'In Warranty' in car_details_list:
            waranty_condition.append('In Warranty')
        else:
            waranty_condition.append('Not available')

        if 'Factory Paint' in car_details_list:
            Car_paint.append('Factory Paint')
        else:
            Car_paint.append('Not available')
        
        
    except:
            car_details.append('Not available')
            
        
    #Getting the phone number before we Move on to another page
    
    try:
        button = driver.find_element_by_xpath('/html/body/app-root/app-index/div/div[2]/app-index/div[3]/div/div/div[2]/div[1]/div/div[2]/div[2]/div[2]')
        button.click()
        #time.sleep(1)
        Mobile_number.append(soup.find('a',{'class':'text-decoration-none text--brand'}).text.strip())
    except:
        Mobile_number.append('Not available')
        
        
    #-----------------------------------------        
    #Section 4: create the data frame 
    #----------------------------------------- 
    
    
    df={'Mobile_number':Mobile_number,
             'Model_name':Model_name,
             'SubModel_name':SubModel_name,
             'Car_condition':Car_condition,
             'Kilometers_crossed':Kilometers_crossed,
             'Car_price':Car_price,
             "Seller_City":Seller_City,
             'Model_year':Model_year,
             'body_type':body_type,
             'Transmission_type':Transmission_type,
             'Motor_capacity':Motor_capacity,
             'Car_color':Car_color,
             'waranty_condition':waranty_condition,
             'Car_paint':Car_paint,
             'Ad_post_date':Ad_post_date,
             'Interior_Features':Interior_Features,
             'Exterior_Features':Exterior_Features,
             'Multimedia_Features':Multimedia_Features,
             'Safety_Features':Safety_Features}
    ADs_list.append(df)
        
    print(f'AD number {count}: Done')
    count += 1

AD number 1: Collecting data....
AD number 1: Done
AD number 2: Collecting data....
AD number 2: Done
AD number 3: Collecting data....
AD number 3: Done
AD number 4: Collecting data....
AD number 4: Done
AD number 5: Collecting data....
AD number 5: Done
AD number 6: Collecting data....
AD number 6: Done
AD number 7: Collecting data....
AD number 7: Done
AD number 8: Collecting data....
AD number 8: Done
AD number 9: Collecting data....
AD number 9: Done
AD number 10: Collecting data....
AD number 10: Done


Done !

We have finished scraping our data completely

In [9]:
#Converting the Collected data into a dataframe
df = pd.DataFrame(df)

In [10]:
#checking results
df

,Mobile_number,Model_name,SubModel_name,Car_condition,Kilometers_crossed,Car_price,Seller_City,Model_year,body_type,Transmission_type,Motor_capacity,Car_color,waranty_condition,Car_paint,Ad_post_date,Interior_Features,Exterior_Features,Multimedia_Features,Safety_Features
0,01005003993,Mercedes,Mercedes E350,مرسيدس E350\n* موديل 2019\n* افاينجارد\n* عداد...,"Used | 45,000 KM","1,300,000 EGP",Maadi | Cairo,2019,Sedan,Automatic,3500 cc,Not available,Not available,Factory Paint,"45,000 KM",Analoge Air ConditionRear parking sensorsRear ...,Sun RoofPower mirrors,AUXBluetoothUSBTouch screen,Traction ControlAnti-Lock Braking System (ABS)...
1,01028001250,Mercedes,Mercedes E300,غندور اوتو - Ghandour Auto\nاختارها صح واستورد...,"Used | 25,996 KM","900,000 EGP",Sheikh Zayed | Giza,2021,Sedan,Automatic,1900 cc,Not available,Not available,Factory Paint,"25,996 KM",Not available,Not available,Not available,Not available
2,01028001250,Mercedes,Mercedes GLC 300,غندور اوتو - Ghandour Auto\nاختارها صح واستورد...,"Used | 2,617 KM","1,340,000 EGP",Sheikh Zayed | Giza,2020,SUV,Automatic,2000 cc,Not available,Not available,Factory Paint,"2,617 KM",Analoge Air ConditionRear parking sensorsRear ...,Sun RoofPanorama RoofPower mirrors,BluetoothUSB,Anti-Lock Braking System (ABS)Cruise ControlAi...
3,01061696665,Mercedes,Mercedes GLA 200,غندور اوتو - Ghandour Auto\nاختارها صح واستورد...,"Used | 2,000 KM","1,050,000 EGP",New Cairo | Cairo,2021,SUV,Automatic,1300 cc,Not available,Not available,Factory Paint,"2,000 KM",Keyless Start/StopAnaloge Air ConditionRear pa...,Power mirrors,AUXBluetoothUSB,Traction ControlAnti-Lock Braking System (ABS)...
4,01061696665,Mercedes,Mercedes E300,غندور اوتو - Ghandour Auto\nاختارها صح واستورد...,"Used | 6,314 KM","1,425,000 EGP",New Cairo | Cairo,2021,Sedan,Automatic,1900 cc,Not available,Not available,Factory Paint,"6,314 KM",Keyless Start/StopAnaloge Air ConditionRear pa...,Sun RoofPanorama RoofPower mirrors,AUXBluetoothUSB,Traction ControlAnti-Lock Braking System (ABS)...
5,01061696665,BMW,BMW 740,استورد عربيتك Ghandour Auto - غندور اوتو\n\n\n...,"Used | 37,000 KM","1,100,000 EGP",New Cairo | Cairo,2019,Sedan,Automatic,2000 cc,Not available,Not available,Factory Paint,"37,000 KM",Not available,Not available,Not available,Not available
6,01061696665,Audi,Audi A6,استورد عربيتك Ghandour Auto - غندور اوتو\n\n\n...,"Used | 19,500 KM","1,500,000 EGP",New Cairo | Cairo,2020,Sedan,Automatic,2000 cc,Not available,Not available,Factory Paint,"19,500 KM",Keyless Start/StopAnaloge Air ConditionRear pa...,Sun RoofPanorama RoofPower mirrors,AUXBluetoothUSB,Anti-Lock Braking System (ABS)Cruise ControlAi...
7,01028001250,BMW,BMW 530i,غندور اوتو - Ghandour Auto\nاختارها صح واستورد...,"Used | 33,622 KM","1,030,000 EGP",Sheikh Zayed | Giza,2020,Sedan,Automatic,2000 cc,Not available,Not available,Factory Paint,"33,622 KM",Not available,Not available,Not available,Not available
8,01028001250,Mercedes,Mercedes GLC 300,غندور اوتو - Ghandour Auto\nاختارها صح واستورد...,"Used | 47,760 KM","1,340,000 EGP",Sheikh Zayed | Giza,2020,SUV,Automatic,2000 cc,Not available,Not available,Factory Paint,"47,760 KM",Analoge Air ConditionRear parking sensorsRear ...,Sun RoofPanorama RoofPower mirrors,BluetoothUSB,Anti-Lock Braking System (ABS)Cruise ControlAi...
9,01061696665,Mercedes,Mercedes E300,غندور اوتو - Ghandour Auto\nاختارها صح واستورد...,"Used | 7,980 KM","1,400,000 EGP",New Cairo | Cairo,2021,Sedan,Automatic,1900 cc,Not available,Not available,Factory Paint,"7,980 KM",Keyless Start/StopAnaloge Air ConditionRear pa...,Sun RoofPanorama RoofPower mirrors,AUXBluetoothUSB,Traction ControlAnti-Lock Braking System (ABS)...


Now Let's Collect all the Additional Features of the cars from the website in a list and and create a column for each feature and mention wether the car has that feature or not 

In [11]:
#collecting all features
All_Features=['Keyless Start/Stop', 'Analoge Air Condition', 'Rear parking sensors', 'Rear View Camera', 'Leather Seats', 'Electronic Window', 'Multi function steering wheel', 'Paddle shifters', 'Front parking sensors', 'Front Camera','Sun Roof','Panorama Roof','Power mirrors','AUX','Bluetooth','USB','Touch screen','Navigation System','Tire Pressure Monitoring (TPMS)','Electronic Stability Program (ESP)','Cruise Control','Anti-Lock Braking System (ABS)','Air Bags For Driver','Air Bags Passenger','Side Airbags','Traction Control']

In [12]:
#Creating a column for each feature in the All_Features list in the dataframe and giving it a basic value ('Not available')
for feature in All_Features:
    df[feature] = 'Not available'

In [13]:
#Checking result
df.head(2)

,Mobile_number,Model_name,SubModel_name,Car_condition,Kilometers_crossed,Car_price,Seller_City,Model_year,body_type,Transmission_type,Motor_capacity,Car_color,waranty_condition,Car_paint,Ad_post_date,Interior_Features,Exterior_Features,Multimedia_Features,Safety_Features,Keyless Start/Stop,Analoge Air Condition,Rear parking sensors,Rear View Camera,Leather Seats,Electronic Window,Multi function steering wheel,Paddle shifters,Front parking sensors,Front Camera,Sun Roof,Panorama Roof,Power mirrors,AUX,Bluetooth,USB,Touch screen,Navigation System,Tire Pressure Monitoring (TPMS),Electronic Stability Program (ESP),Cruise Control,Anti-Lock Braking System (ABS),Air Bags For Driver,Air Bags Passenger,Side Airbags,Traction Control
0,01005003993,Mercedes,Mercedes E350,مرسيدس E350\n* موديل 2019\n* افاينجارد\n* عداد...,"Used | 45,000 KM","1,300,000 EGP",Maadi | Cairo,2019,Sedan,Automatic,3500 cc,Not available,Not available,Factory Paint,"45,000 KM",Analoge Air ConditionRear parking sensorsRear ...,Sun RoofPower mirrors,AUXBluetoothUSBTouch screen,Traction ControlAnti-Lock Braking System (ABS)...,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available
1,01028001250,Mercedes,Mercedes E300,غندور اوتو - Ghandour Auto\nاختارها صح واستورد...,"Used | 25,996 KM","900,000 EGP",Sheikh Zayed | Giza,2021,Sedan,Automatic,1900 cc,Not available,Not available,Factory Paint,"25,996 KM",Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available


Great! Now I can move on to the final step

I will create a function that checks each value in the All_Features List and Return the value of the feature in the feature column if it found it in the car, if it didn't find it it will leave the value as Not available

In [14]:
def Capture_features(x,b,c,d):
    for i in range(len(df[x])):
        for s in All_Features:
            if s in df[x][i]:
                df[s][i]=s
            if s in df[b][i]:
                df[s][i]=s
            if s in df[c][i]:
                df[s][i]=s
            if s in df[d][i]:
                df[s][i]=s

In [15]:
#Applying the function on the collected features for each car
Capture_features('Interior_Features','Exterior_Features','Multimedia_Features','Safety_Features')

In [16]:
#Checking final result
df

,Mobile_number,Model_name,SubModel_name,Car_condition,Kilometers_crossed,Car_price,Seller_City,Model_year,body_type,Transmission_type,Motor_capacity,Car_color,waranty_condition,Car_paint,Ad_post_date,Interior_Features,Exterior_Features,Multimedia_Features,Safety_Features,Keyless Start/Stop,Analoge Air Condition,Rear parking sensors,Rear View Camera,Leather Seats,Electronic Window,Multi function steering wheel,Paddle shifters,Front parking sensors,Front Camera,Sun Roof,Panorama Roof,Power mirrors,AUX,Bluetooth,USB,Touch screen,Navigation System,Tire Pressure Monitoring (TPMS),Electronic Stability Program (ESP),Cruise Control,Anti-Lock Braking System (ABS),Air Bags For Driver,Air Bags Passenger,Side Airbags,Traction Control
0,01005003993,Mercedes,Mercedes E350,مرسيدس E350\n* موديل 2019\n* افاينجارد\n* عداد...,"Used | 45,000 KM","1,300,000 EGP",Maadi | Cairo,2019,Sedan,Automatic,3500 cc,Not available,Not available,Factory Paint,"45,000 KM",Analoge Air ConditionRear parking sensorsRear ...,Sun RoofPower mirrors,AUXBluetoothUSBTouch screen,Traction ControlAnti-Lock Braking System (ABS)...,Not available,Analoge Air Condition,Rear parking sensors,Rear View Camera,Leather Seats,Electronic Window,Not available,Not available,Not available,Not available,Sun Roof,Not available,Power mirrors,AUX,Bluetooth,USB,Touch screen,Not available,Not available,Not available,Cruise Control,Anti-Lock Braking System (ABS),Air Bags For Driver,Air Bags Passenger,Side Airbags,Traction Control
1,01028001250,Mercedes,Mercedes E300,غندور اوتو - Ghandour Auto\nاختارها صح واستورد...,"Used | 25,996 KM","900,000 EGP",Sheikh Zayed | Giza,2021,Sedan,Automatic,1900 cc,Not available,Not available,Factory Paint,"25,996 KM",Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available
2,01028001250,Mercedes,Mercedes GLC 300,غندور اوتو - Ghandour Auto\nاختارها صح واستورد...,"Used | 2,617 KM","1,340,000 EGP",Sheikh Zayed | Giza,2020,SUV,Automatic,2000 cc,Not available,Not available,Factory Paint,"2,617 KM",Analoge Air ConditionRear parking sensorsRear ...,Sun RoofPanorama RoofPower mirrors,BluetoothUSB,Anti-Lock Braking System (ABS)Cruise ControlAi...,Not available,Analoge Air Condition,Rear parking sensors,Rear View Camera,Leather Seats,Electronic Window,Not available,Not available,Not available,Not available,Sun Roof,Panorama Roof,Power mirrors,Not available,Bluetooth,USB,Not available,Not available,Not available,Not available,Cruise Control,Anti-Lock Braking System (ABS),Air Bags For Driver,Air Bags Passenger,Not available,Not available
3,01061696665,Mercedes,Mercedes GLA 200,غندور اوتو - Ghandour Auto\nاختارها صح واستورد...,"Used | 2,000 KM","1,050,000 EGP",New Cairo | Cairo,2021,SUV,Automatic,1300 cc,Not available,Not available,Factory Paint,"2,000 KM",Keyless Start/StopAnaloge Air ConditionRear pa...,Power mirrors,AUXBluetoothUSB,Traction ControlAnti-Lock Braking System (ABS)...,Keyless Start/Stop,Analoge Air Condition,Rear parking sensors,Rear View Camera,Leather Seats,Electronic Window,Not available,Not available,Not available,Not available,Not available,Not available,Power mirrors,AUX,Bluetooth,USB,Not available,Not available,Not available,Not available,Cruise Control,Anti-Lock Braking System (ABS),Air Bags For Driver,Air Bags Passenger,Side Airbags,Traction Control
4,01061696665,Mercedes,Mercedes E300,غندور اوتو - Ghandour Auto\nاختارها صح واستورد...,"Used | 6,314 KM","1,425,000 EGP",New Cairo | Cairo,2021,Sedan,Automatic,1900 cc,Not available,Not available,Factory Paint,"6,314 KM",Keyless Start/StopAnaloge Air ConditionRear pa...,Sun RoofPanorama RoofPower mirrors,AUXBluetoothUSB,Traction ControlAnti-Lock Bra

Amazing! Our data seems Consistent.

Now i can export it to an excel file and move on through the stages of my project.

In [ ]:
#exporting data into an excel file
#df.to_excel('Contactcars.com.xlsx')